# SolarMED model

Model that integrates all the component models and subproblems into one interface.

This notebook is used to develop and test it

In [9]:
from pathlib import Path
import time
import datetime
import numpy as np
import pandas as pd
from iapws import IAPWS97 as w_props
import hjson
from loguru import logger

# Visualization packages
from phd_visualizations import save_figure
from phd_visualizations.constants import generate_plotly_config
from phd_visualizations.test_timeseries import experimental_results_plot

from solarmed_modeling.utils import data_preprocessing, data_conditioning

# auto reload modules
%load_ext autoreload
%autoreload 2

logger.disable("phd_visualizations.utils.units")

# Paths definition
output_path: Path = Path("../../docs/models/attachments")
data_path: Path = Path("../data")

date_str: str = 20231106 # "20230511"  # '20230630'
filename_process_data = f"{date_str}_solarMED.csv"
filename_process_data2 = f"{date_str}_MED.csv"

# Available data to test
# data/calibration/20230807_aquasol.csv
# data/calibration/20230707_20230710_datos_tanques.csv
# Nextcloud/Juanmi_MED_PSA/EURECAT/data/20231030_solarMED.csv

sample_rate = "400s"
sample_rate_numeric = int(sample_rate[:-1])

# Resample figures using plotly_resampler
resample_figures: bool = False

test_model_dumping_during_runtime: bool = False # Test initializing the model instance at every step using the dump from the previous one
test_partial_init_at_runtime: bool = False # Test initializing the model instance at every step as an initial initialization just giving the internal FSM states and subsystem states (tanks temperatures, etc) from the previous step
feedback_experimental_data_at_runtime: bool = False # Partial initialization with feedback of experimental data instead of previous step data

assert sum([test_model_dumping_during_runtime, test_partial_init_at_runtime, feedback_experimental_data_at_runtime]) <= 1, "Only one test can be selected" 


# Parameters
cost_w: float = 3  # €/m³, cost of water
cost_e: float = 0.05  # €/kWh, cost of electricity


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Pre-processing

In [10]:
data_paths = [
    data_path / f"datasets/{filename_process_data}",
    data_path / f"datasets/{filename_process_data2}",
]

with open(data_path / "variables_config.hjson") as f:
    vars_config = hjson.load(f)

with open(data_path / "plot_config.hjson") as f:
    plot_config = hjson.load(f)

# Load data and preprocess data
df = data_preprocessing(data_paths, vars_config, sample_rate_key=sample_rate)

# Condition data
df = data_conditioning(
    df,
    cost_w=cost_w,
    cost_e=cost_e,
    sample_rate_numeric=sample_rate_numeric,
    vars_config=vars_config,
)

# Test visualization
with open(data_path / "plot_config.hjson") as f:
    plot_config = hjson.load(f)

fig = experimental_results_plot(
    plot_config, df, vars_config=vars_config, resample=resample_figures
)

fig.show(
    config=generate_plotly_config(
        fig, figure_name=f'solar_med_{df.index[0].strftime("%Y%m%d")}'
    )
)


2025-02-20 15:29:56.483 | INFO     | solarmed_modeling.utils:data_preprocessing:113 - Reading data from 20231106_solarMED.csv


2025-02-20 15:29:56.654 | DEBUG    | solarmed_modeling.utils:process_dataframe:69 - Index([], dtype='object')
2025-02-20 15:29:56.655 | INFO     | solarmed_modeling.utils:process_dataframe:76 - Number of duplicate index values in df: 0
2025-02-20 15:29:56.661 | INFO     | solarmed_modeling.utils:data_preprocessing:133 - Reading data from 20231106_MED.csv
2025-02-20 15:29:56.929 | DEBUG    | solarmed_modeling.utils:process_dataframe:69 - Index([], dtype='object')
2025-02-20 15:29:56.931 | INFO     | solarmed_modeling.utils:process_dataframe:76 - Number of duplicate index values in df: 0
2025-02-20 15:29:56.934 | DEBUG    | solarmed_modeling.utils:data_preprocessing:152 - Common columns in both dataframes: Index(['FT-AQU-100', 'TT-AQU-107a', 'CT-DES-001', 'FT-DES-003', 'FT-DES-005',
       'SW2TC1', 'TE-DES-015', 'TT-DES-030'],
      dtype='object'), dropping them from the auxiliary dataframe.
2025-02-20 15:29:56.947 | WARNING  | phd_visualizations.utils:rename_signal_ids_to_var_ids:81 -

## Evaluation

In [32]:
# Test model
import math
from solarmed_modeling.solar_med import SolarMED, FsmParameters, FixedModelParameters
from solarmed_modeling.fsms.med import FsmParameters as MedFsmParams
from solarmed_modeling.fsms.sfts import FsmParameters as SftsFsmParams

logger.enable("solarmed_modeling")

fsm_params: FsmParameters = FsmParameters(
    med=MedFsmParams(
        vacuum_duration_time=1 * 3600,  # 1 hour
        brine_emptying_time=5 * 60,  # 30 minutes
        startup_duration_time=5 * 60,  # 20 minutes
        off_cooldown_time=0 * 3600,  # 12 hours
        active_cooldown_time=0 * 3600,  # 3 hours
    ),
    sf_ts=SftsFsmParams(
        recirculating_ts_enabled=False,
        idle_cooldown_time=1 * 3600,  # 1 hour
    ),
)

fixed_model_params = FixedModelParameters()
fixed_model_params.sf.Tmax = 120
fixed_model_params.med.qmed_c_min = 3

span = math.ceil(600 / sample_rate_numeric)  # 600 s
idx_start = np.max([span, 2])  # idx_start-1 should at least be one
idx_end = len(df)

# Initialize model
model: SolarMED = SolarMED(
    resolution_mode="constant-water-props",
    use_models=True,
    use_finite_state_machine=True,
    sample_time=sample_rate_numeric,
    fsms_params=fsm_params,
    fixed_model_params=fixed_model_params,
    # Initial states
    ## Thermal storage
    Tts_h=[
        df["Tts_h_t"].iloc[idx_start],
        df["Tts_h_m"].iloc[idx_start],
        df["Tts_h_b"].iloc[idx_start],
    ],
    Tts_c=[
        df["Tts_c_t"].iloc[idx_start],
        df["Tts_c_m"].iloc[idx_start],
        df["Tts_c_b"].iloc[idx_start],
    ],
    ## Solar field
    Tsf_in_ant=df["Tsf_in"].iloc[idx_start - span : idx_start].values,
    qsf_ant=df["qsf"].iloc[idx_start - span : idx_start].values,
    # cost_w = 3, # €/m³
    # cost_e = 0.05, # €/kWhe,
)

assert (
    model.fsms_params.med.vacuum_duration_time == fsm_params.med.vacuum_duration_time
), "wat"


2025-02-20 16:06:14.359 | INFO     | solarmed_modeling.solar_med:model_post_init:625 - 
        SolarMED model initialized with: 
            - Evaluating models: True
            - Evaluating finite state machines: True
            - Resolution mode: constant-water-props
            - On limits violation policy: clip
            - Sample time: 400.0 s
            - MED actuators: ['med_brine_pump', 'med_feed_pump', 'med_distillate_pump', 'med_cooling_pump', 'med_heatsource_pump']
            - Solar field actuators: ['sf_pump']
            - Thermal storage actuators: ['ts_src_pump']
            - Model parameters: ModelParameters(sf=ModelParameters(beta=0.0436396, H=13.676448551722462, gamma=0.1), ts=ModelParameters(UA_h=(0.0069818, 0.00584034, 0.03041486), V_h=(5.94771006, 4.87661781, 2.19737023), UA_c=(0.01396848, 0.0001, 0.02286885), V_c=(5.33410037, 7.56470594, 0.90547187)), hex=ModelParameters(UA=13536.596, H=0.0))
            - Fixed model parameters: FixedModelParameters(med=F

### Model exporting/importing


In [33]:
# Save model configuration
model_config = model.model_dump_configuration()

# Export model instance and recreate a new instance
dumped_model_instance: dict = model.dump_instance()

assert (
    "Tts_h" in dumped_model_instance
), "Missing INITIAL_STATES variables in dumped instance"

# display(dumped_model_instance)
model: SolarMED = SolarMED(**dumped_model_instance)


2025-02-20 16:06:18.595 | INFO     | solarmed_modeling.solar_med:model_post_init:625 - 
        SolarMED model initialized with: 
            - Evaluating models: True
            - Evaluating finite state machines: True
            - Resolution mode: constant-water-props
            - On limits violation policy: clip
            - Sample time: 400.0 s
            - MED actuators: ['med_brine_pump', 'med_feed_pump', 'med_distillate_pump', 'med_cooling_pump', 'med_heatsource_pump']
            - Solar field actuators: ['sf_pump']
            - Thermal storage actuators: ['ts_src_pump']
            - Model parameters: ModelParameters(sf=ModelParameters(beta=0.0436396, H=13.676448551722462, gamma=0.1), ts=ModelParameters(UA_h=(0.0069818, 0.00584034, 0.03041486), V_h=(5.94771006, 4.87661781, 2.19737023), UA_c=(0.01396848, 0.0001, 0.02286885), V_c=(5.33410037, 7.56470594, 0.90547187)), hex=ModelParameters(UA=13536.596, H=0.0))
            - Fixed model parameters: FixedModelParameters(med=F

### Model evaluation

In [34]:
from dataclasses import asdict

model: SolarMED = SolarMED(**dumped_model_instance)

df_mod = model.to_dataframe()
fsm_internal_states = []

# Run model
for idx in range(idx_start + 1, idx_end):
    ds = df.iloc[idx]

    # logger.info(f"Iteration {idx} / {idx_end}")
    start_time = time.time()
    
    if test_model_dumping_during_runtime:
        model = SolarMED(**model.dump_instance())
        
    elif test_partial_init_at_runtime:
        model_instance = model.dump_instance()
        model_instance.update(dict(
            # Initial states
            ## FSM states
            fsms_internal_states=model.fsms_internal_states,
            med_state=model.med_state,
            sf_ts_state=model.sf_ts_state,
            # fsms_internal_states=FsmInternalState(**asdict(model.fsms_internal_states)),
            ## Thermal storage
            Tts_h=model.Tts_h,
            Tts_c=model.Tts_c,
            ## Solar field
            Tsf_in_ant=model.Tsf_in_ant,
            qsf_ant=model.qsf_ant,
        ))
        model = SolarMED(**model_instance)
        
    elif feedback_experimental_data_at_runtime:
        model_instance = model.dump_instance()
        model_instance.update(dict(
            # Initial states
            ## FSM states
            fsms_internal_states=model.fsms_internal_states, # From previous step
            med_state=model.med_state,
            sf_ts_state=model.sf_ts_state,
            ## Thermal storage
            Tts_h=[
                ds["Tts_h_t"],
                ds["Tts_h_m"],
                ds["Tts_h_b"],
            ],
            Tts_c=[
                ds["Tts_c_t"],
                ds["Tts_c_m"],
                ds["Tts_c_b"],
            ],
            ## Solar field
            Tsf_in_ant=df["Tsf_in"].iloc[idx - span : idx].values,
            qsf_ant=df["qsf"].iloc[idx - span : idx].values,
        ))
        model = SolarMED(**model_instance)

    model.step(
        # Decision variables
        ## MED
        qmed_s=ds["qmed_s"],
        qmed_f=ds["qmed_f"],
        Tmed_s_in=ds["Tmed_s_in"],
        Tmed_c_out=ds["Tmed_c_out"],
        ## Thermal storage
        qts_src=ds["qhx_s"],
        ## Solar field
        qsf=ds["qsf"],
        med_vacuum_state=2,
        # Environment variables
        Tmed_c_in=ds["Tmed_c_in"],
        Tamb=ds["Tamb"],
        I=ds["I"],
    )
    
    fsm_internal_states.append(asdict(model.fsms_internal_states))

    logger.info(
        f"Finished Iteration {idx} / {idx_end} - {df.index[idx]:%H:%M:%S}, elapsed time: {time.time()-start_time:.2f} seconds."
    )

    df_mod = model.to_dataframe(df_mod)


2025-02-20 16:06:22.016 | INFO     | solarmed_modeling.solar_med:model_post_init:625 - 
        SolarMED model initialized with: 
            - Evaluating models: True
            - Evaluating finite state machines: True
            - Resolution mode: constant-water-props
            - On limits violation policy: clip
            - Sample time: 400.0 s
            - MED actuators: ['med_brine_pump', 'med_feed_pump', 'med_distillate_pump', 'med_cooling_pump', 'med_heatsource_pump']
            - Solar field actuators: ['sf_pump']
            - Thermal storage actuators: ['ts_src_pump']
            - Model parameters: ModelParameters(sf=ModelParameters(beta=0.0436396, H=13.676448551722462, gamma=0.1), ts=ModelParameters(UA_h=(0.0069818, 0.00584034, 0.03041486), V_h=(5.94771006, 4.87661781, 2.19737023), UA_c=(0.01396848, 0.0001, 0.02286885), V_c=(5.33410037, 7.56470594, 0.90547187)), hex=ModelParameters(UA=13536.596, H=0.0))
            - Fixed model parameters: FixedModelParameters(med=F

In [35]:
# Sync model index with measured data
df_mod.index = df.index[
    idx_start : idx if idx < idx_end - 1 else idx_end
]  # idx_start-1 because now we are adding one element after the initialization

# Update plot config
with open(data_path / "plot_config_validation.hjson") as f:
    plt_config = hjson.load(f)

fig = experimental_results_plot(
    plt_config, df, df_comp=df_mod, vars_config=vars_config, resample=resample_figures
)

fig.show(
    config=generate_plotly_config(
        fig, figure_name=f'solar_med_{df.index[0].strftime("%Y%m%d")}'
    )
)


##### Run without re-initialization

```json
{'ITAE': np.float64(324709.6889793461),
 'ISE': np.float64(52867.176381307334),
 'IAE': np.float64(5570.5631931096395),
 'RMSE': np.float64(7.8726064585820925),
 'MAE': np.float64(6.530554739870621),
 'MSE': np.float64(61.97793245170848),
 'R2': np.float64(0.5174667756097235)}
```

#### Run with re-initialization from dump

```json
{'ITAE': np.float64(322923.44723907113),
 'ISE': np.float64(52354.85891568201),
 'IAE': np.float64(5539.583512325931),
 'RMSE': np.float64(7.8343682457304284),
 'MAE': np.float64(6.49423623953802),
 'MSE': np.float64(61.37732580970927),
 'R2': np.float64(0.5221428376868169)}
```

##### Run with equivalent partial re-initialization

```json
{'ITAE': np.float64(322923.44723907113),
 'ISE': np.float64(52354.85891568201),
 'IAE': np.float64(5539.583512325931),
 'RMSE': np.float64(7.8343682457304284),
 'MAE': np.float64(6.49423623953802),
 'MSE': np.float64(61.37732580970927),
 'R2': np.float64(0.5221428376868169)}
```

For some reason, partial initializations provide similar but slightly different
(better) results than the normal case. They should be exactly the same since
the difference mode of initialization or just reusing the instance should not
affect the results. Anyway, the difference is minimal so for now moving on (20241203)

##### Run with partial re-initialization and experimental data feedback

As expected, this provides by far the best results, since the model only needs
to predict one step and then is feed the actual values for the next one.

```json
{'ITAE': np.float64(49998.030406532496),
 'ISE': np.float64(3921.8318949493278),
 'IAE': np.float64(1045.5315758381903),
 'RMSE': np.float64(2.1442231053728262),
 'MAE': np.float64(1.2257111088372687),
 'MSE': np.float64(4.597692725614687),
 'R2': np.float64(0.9642043642327824)}
```

In [15]:
from solarmed_modeling.metrics import calculate_metrics
from solarmed_modeling.solar_med.utils import out_var_ids

metrics = calculate_metrics(
    df_mod[out_var_ids].values, df.iloc[idx_start:][out_var_ids].values
)

metrics


{'ITAE': 326731.71285092825,
 'ISE': 53515.12139578811,
 'IAE': 5615.1353240114295,
 'RMSE': 7.920703235013228,
 'MAE': 6.582808117246693,
 'MSE': 62.73753973714902,
 'R2': 0.5115528036810929}

In [16]:
# Save figure
# save_figure(
#     figure_name=f"SolarMED_validation_{df.index[0].strftime('%Y%m%d')}",
#     figure_path=output_path,
#     fig=fig, formats=('png', 'html'),
#     width=fig.layout.width, height=fig.layout.height, scale=2
# )


In [26]:
from solarmed_modeling.visualization.fsm.state_evolution import plot_episode_state_evolution
from solarmed_modeling.fsms import SfTsState, MedState

fig = plot_episode_state_evolution(df_mod, subsystems_state_cls=[SfTsState, MedState], show_edges=False, width=1000)

fig


2025-02-11 09:42:50.115 | WARNING  | solarmed_modeling.visualization.fsm.state_evolution:plot_state_graph:85 - There are more samples than the maximum specified (30), states will be shown every 4 samples. Aliasing may occur


FigureWidget({
    'data': [{'hoverinfo': 'text',
              'line': {'color': 'rgb(50,50,50)', 'width': 0.5},
              'marker': {'color': '#ff7800', 'size': 20, 'symbol': 'circle-dot'},
              'mode': 'markers',
              'name': 'states',
              'showlegend': False,
              'text': array(['IDLE', 'HEATING_UP_SF', 'SF_HEATING_TS', ..., 'HEATING_UP_SF',
                             'SF_HEATING_TS', 'RECIRCULATING_TS'], dtype=object),
              'type': 'scatter',
              'uid': '6dc4abf4-227f-4f37-b3f7-8c100c33c8d5',
              'x': [0, 0, 0, 0, 4, 4, 4, 4, 8, 8, 8, 8, 12, 12, 12, 12, 16, 16,
                    16, 16, 20, 20, 20, 20, 24, 24, 24, 24, 28, 28, 28, 28, 32, 32,
                    32, 32, 36, 36, 36, 36, 40, 40, 40, 40, 44, 44, 44, 44, 48, 48,
                    48, 48, 52, 52, 52, 52, 56, 56, 56, 56, 60, 60, 60, 60, 64, 64,
                    64, 64, 68, 68, 68, 68, 72, 72, 72, 72, 76, 76, 76, 76, 80, 80,
                   

### Model evaluation using `evaluate_model`

In [11]:
from solarmed_modeling.solar_med.utils import evaluate_model
from solarmed_modeling.solar_med import ModelParameters, FixedModelParameters

logger.enable("solarmed_modeling.solar_med.utils")

dfs_mod, stats = evaluate_model(
    df=df,
    sample_rate=sample_rate_numeric,
    model_params=ModelParameters(),
    fixed_model_params=FixedModelParameters(),
    alternatives_to_eval=["constant-water-props"],
)

fig = experimental_results_plot(
    plot_config,
    df,
    df_comp=dfs_mod[0],
    vars_config=vars_config,
    resample=resample_figures,
)

fig.show(
    config=generate_plotly_config(
        fig, figure_name=f'solar_med_{df.index[0].strftime("%Y%m%d")}'
    )
)


2024-11-21 14:12:08.451 | INFO     | solarmed_modeling.solar_med.utils:evaluate_model:56 - Starting evaluation of alternative constant-water-props. Sample rate = 400 s
2024-11-21 14:12:09.111 | INFO     | solarmed_modeling.solar_med:init_matlab_engine:1461 - MATLAB engine initialized
2024-11-21 14:12:09.116 | INFO     | solarmed_modeling.solar_med:model_post_init:563 - 
        SolarMED model initialized with: 
            - Evaluating models: True
            - Evaluating finite state machines: True
            - Resolution mode: constant-water-props
            - On limits violation policy: clip
            - Sample time: 400.0 s
            - MED actuators: ['med_brine_pump', 'med_feed_pump', 'med_distillate_pump', 'med_cooling_pump', 'med_heatsource_pump']
            - Solar field actuators: ['sf_pump']
            - Thermal storage actuators: ['ts_src_pump']
            - Model parameters: ModelParameters(sf=ModelParameters(beta=0.0436396, H=13.676448551722462, gamma=0.1), ts=Mod